In [1]:
#Used for card shuffle
import random

#boolean to know if hand is in play
playing = False

chip_pool = 100


bet = 1

#Spades, Diamonds, Hearts, Clubs
suits = ('S','D','H','C')

#Possible Card Ranks
ranking = ('A','2','3','4','5','6','7','8','9','10','J','Q','K')

#Point equivalents to card values. Ace can also count as 11 so needs to be addressed later
card_value = {'A':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'10':10,'J':10,'Q':10,'K':10}

restart_phrase = "Press 'd' to deal out the cards again or press 'q' to quit."

In [2]:
class Card:
    def __init__(self, suit, rank):
        self.suit = suit
        self.rank = rank
    
    def __str__(self):
        return self.suit + self.rank
    
    def grab_suit(self):
        return self.suit
    
    def grab_rank(self):
        return self.rank
    
    def draw(self):
        print (self.suit + self.rank)

In [3]:
class Hand:
    def __init__(self):
        self.cards = []
        self.value = 0
        # Aces can be 1 or 11 so need to define it here
        self.ace = False 
    
    def __str__(self):
        #return a string of current hand composition
        hand_comp = ""
        for card in self.cards:
            card_name = card.__str__()
            hand_comp += " " + card_name
        
        return "The hand has %s" %hand_comp
    
    def add_card(self, card):
        #add a card to the hand
        self.cards.append(card)
        
        #check for aces
        if card.rank == 'A':
            self.ace = True
        self.value += card_value[card.rank]
        
    def calc_value(self):
        #calculate the value of the hand, make aces an 11 if they don't bust the hand
        if (self.ace == True and self.value < 12):
            return self.value + 10
        else:
            return self.value
        
    def draw(self, hidden):
        if hidden == True and playing == True:
        #don't show first hidden card
            starting_card = 1
        else:
            starting_card = 0
        for x in range(starting_card, len(self.cards)):
            self.cards[x].draw()
    

In [4]:
class Deck:
    
    def __init__(self):
        self.deck = []
        for suit in suits:
            for rank in ranking:
                self.deck.append(Card(suit, rank))
                
    def shuffle(self):
        random.shuffle(self.deck)
        
    def deal(self):
        single_card = self.deck.pop()
        return single_card
    
    def __str__(self):
        deck_comp = ""
        for card in self.cards:
            deck_comp += " " + deck_comp.__str__()
        return "The deck has " + deck_comp
        

In [5]:
def make_bet():
    
    global bet
    bet = 0
    
    print "What amount of chips would you like to bet? (Whole integers please.)"
        
    while bet == 0:
        bet_comp = raw_input()
        bet_comp = int(bet_comp)
        if bet_comp >= 1 and bet_comp <= chip_pool:
            bet = bet_comp
        else:
            print "Invalid bet. You only have " + str(chip_pool) + " remaining"
            
    
   

In [6]:
def deal_cards():
    
    #set up all local variables
    global result, playing, deck, player_hand, dealer_hand, chip_pool, bet
    
    #create a deck
    deck = Deck()
    
    #shuffle the deck
    deck.shuffle()
    
    #set up bet
    make_bet()
    
    #set up both player and dealer hands
    player_hand = Hand()
    dealer_hand = Hand()
    
    #deal out initial cards
    player_hand.add_card(deck.deal())
    player_hand.add_card(deck.deal())
    
    dealer_hand.add_card(deck.deal())
    dealer_hand.add_card(deck.deal())
    
    result = "Hit or Stand? Press h or s: "
    
    if playing == True:
        print "Fold, Sorry."
        chip_pool -= bet
    
    playing = True
    
    game_step()

In [7]:
def hit():
    
    global playing, chip_pool, deck, player_hand, dealer_hand, result, bet
    
    #if hand is in play add card
    if playing:
        if player_hand.calc_value <= 21:
            player_hand.add_card(deck.deal())
        
        print "Players hand is %s"  %player_hand
        
        if player_hand.calc_value > 21:
            result = "Busted! " + restart_phrase
            
            chip_pool -= bet
            playing = False
            
            
    else:
        result = "Sorry, can't hit. " + restart_phrase

    game_step()        

In [8]:
def stand():
    global playing, chip_pool, deck, player_hand, dealer_hand, result, bet
    #this function will play the dealers hand since stand was chosen
    
    if playing == False:
        if player_hand.calc_value() > 0:
            result = "Sorry, you can't stand!"
        
    #now go through all other possible actions
    else:
        
        #soft seventeen rule
        while dealer_hand.calc_value() < 17:
            dealer_hand.add_card(deck.deal())
            
            
        #dealer busts
        if dealer_hand.calc_value > 21:
            result = "Dealer busts, you win! "+ restart_phrase
            chip_pool += bet
            playing == False
        
        #player has better hand then dealer
        elif player_hand.calc_value() > dealer_hand.calc_value():
            result = "Congratulations you beat the dealer!" + restart_phrase
            chip_pool += bet
            playing == False
        
        #hands are tied
        elif player_hand.calc_value() == dealer_hand.calc_value():
            result = "Tied up, Push " + restart_phrase
            playing == False
            
        #dealer beat player
        else:
            result = "Sorry but the dealer has won! " + restart_phrase
            chip_pool -= bet
            playing == False
            
    game_step()      

In [9]:
def game_step():
    #function to print game step/status on output
    
    #display player hand
    print ""
    print('Player hand is: '),
    player_hand.draw(hidden =False)
    
    print('Player hand total is: ') + str(player_hand.calc_value())
    
    #display dealer hand
    print ('Dealer hand is: '),
    dealer_hand.draw(hidden =True)
    
    #if game round is over
    if playing == False:
        print "--- for a total of " + str(dealer_hand.calc_value())
        print "Chip total: " + str(chip_pool)
    #otherwise, don't know the second card yet
    else:
        print "With another card hidden upside down."
        
    #print result of hit or stand
    print result

    player_input()

In [10]:
def game_exit():
    print "Thanks for playing!"
    exit()
    

In [11]:
def player_input():
    plin = raw_input().lower()
    
    if plin == 'h':
        hit()
    elif plin == 's':
        stand()
    elif plin == 'd':
        deal_cards()
    elif plin == 'q':
        game_exit()
    else:
        print "Invalid input...enter h, s, d, or q: "
        player_input()

In [12]:
def intro():
    statement = '''Welcome to BlackJack! Get as close to 21 as you can without going over!
    Dealer hits until she reaches 17. Aces count as 1 or 11.
    Card output goes a letter followed by a number of face notation'''
    print statement

In [13]:
#create a deck
deck = Deck()
#shuffle it
deck.shuffle()
#create player and dealer hand
player_hand = Hand()
dealer_hand = Hand()
#print intro
intro()
#deal out the cards and start the game
deal_cards()

Welcome to BlackJack! Get as close to 21 as you can without going over!
    Dealer hits until she reaches 17. Aces count as 1 or 11.
    Card output goes a letter followed by a number of face notation
What amount of chips would you like to bet? (Whole integers please.)
6

Player hand is:  D5
S10
Player hand total is: 15
Dealer hand is:  HQ
With another card hidden upside down.
Hit or Stand? Press h or s: 
h
Players hand is The hand has  D5 S10

Player hand is:  D5
S10
Player hand total is: 15
Dealer hand is:  D6
HQ
--- for a total of 16
Chip total: 94
Busted! Press 'd' to deal out the cards again or press 'q' to quit.
q
Thanks for playing!
